# RT2 Assignment

## Goal
Starting from the second assignment of the course, create a jupyter notebook to replace the user interface (the node «A»).
Try using widgets to let the user know :
- the position of the robot and all targets that have been set and cancelled in the environment
- the distance of the closest obstacle

## Procedure
First of all we import all the necessary libraries:

In [ ]:
import jupyros as jr
import rospy
import math
import actionlib
import actionlib.msg
import assignment_2_2022.msg
from std_srvs.srv import *
import time
import sys
import select
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from assignment_2_2022.msg import Pos_vel
import ipywidgets as widgets
from assignment_2_2022.srv import Goals_rc, Goals_rcRequest
from sensor_msgs.msg import LaserScan
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
%matplotlib widget

Then we define the callback function for the subscriber, in which we create also the custom message to publish:

In [ ]:
def clbk_odom(msg):
    global last_odom
    # Get the position from the msg
    position_ = msg.pose.pose.position
    # Get the twist from the msg
    vel_lin = msg.twist.twist.linear
    # Create custom message
    pos_vel = Pos_vel()
    pos_vel.x = position_.x
    pos_vel.y = position_.y
    pos_vel.vel_x = vel_lin.x
    pos_vel.vel_y = vel_lin.y
    # Publish the custom message
    pub.publish(pos_vel)
    # Print the position every 100 ms
    curr_t = time.time() * 1000
    if curr_t - last_odom > 100:
        print("\rRobot position: x={}, y={}".format(position_.x, position_.y), end='')
        last_odom = curr_t

Then we define the callback function for the laser scan:

In [ ]:
def clbk_laser(scan):
    global last_laser
    # Only consider obstacles in a 180° field of view in front of the robot
    start_index = len(scan.ranges) // 2 - (len(scan.ranges) // 4)
    end_index = len(scan.ranges) // 2 + (len(scan.ranges) // 4)
    ranges = scan.ranges[start_index:end_index]
    min_dist = min(ranges)
    # Print the distance every 100 ms
    curr_t = time.time() * 1000
    if curr_t - last_laser > 100:
        print("\rClosest obstacle distance: {}".format(min_dist), end='')
        last_laser = curr_t

Now we write the function to call the service to print the number of reached and cancelled goals:

In [ ]:
def call_service():
    rospy.wait_for_service('/goals')
    goals_service = rospy.ServiceProxy('/goals', Goals_rc)
    response = goals_service(Goals_rcRequest())
    return response

Here we create the widgets to input the desired position of the robot or to cancel the goal:

In [ ]:
# Numeric widgets for the position
x_widg = widgets.BoundedFloatText(
    value=0.0,
    description='X:',
    min=-9.0,
    max=9.0,
    style={'description_width': 'initial'},
    step = 0.1,
    layout=widgets.Layout(width='100px')
)
y_widg = widgets.BoundedFloatText(
    value=0.0,
    description='Y:',
    min=-9.0,
    max=9.0,
    style={'description_width': 'initial'},
    step = 0.1,
    layout=widgets.Layout(width='100px')
)

# Button to send the goal position
send_button = widgets.Button(
    value=False,
    description='Send goal',
    disabled=False,
    button_style=''
)

# Button to cancel the goal
cancel_button = widgets.Button(
    value=False,
    description='Cancel goal',
    disabled=False,
    button_style=''
)

Then we have to create the callback functions related to the buttons:

In [ ]:
def on_send_clicked(b):
    global goal, cancelled
    # Create the goal reading the values of x and y of the widgets
    goal = assignment_2_2022.msg.PlanningGoal()
    goal.target_pose.pose.position.x = x_widg.value
    goal.target_pose.pose.position.y = y_widg.value
    # Send the goal to the action server
    client.send_goal(goal)
    # Set the cancelled flag to false
    cancelled = False

In [ ]:
def on_cancel_clicked(b):
    global cancelled
    # Cancel the goal
    client.cancel_goal()
    goal = None
    # Set the cancelled flag to true
    cancelled = True

After we created the callback functions, we have to attach them to the buttons click events:

In [ ]:
send_button.on_click(on_send_clicked)
cancel_button.on_click(on_cancel_clicked)

For visualizing the position of the robot, we can use an animation class:

In [ ]:
class PositionVisualizer:
    def __init__(self):
        # Init function
        self.fig, self.ax = plt.subplots()
        # Settings for robot's position plot
        self.ln, = plt.plot([], [], 'bo', label='Robot position')
        # Settings for target's position plot
        self.goal_ln, = plt.plot([], [], 'r*', markersize=10, label='Goal position')
        # Robot's position data arrays
        self.x_data, self.y_data = [], []
    
    def plot_init(self):
        # Set axis limits
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        # Set the grid
        self.ax.grid(True, color='lightgrey')
        # Set the title
        self.ax.set_title('Robot position')
        # Set the legend
        self.ax.legend(loc='upper left')
        
        return self.ln, self.goal_ln
    
    def odom_callback(self, msg):
        # Callback function to update the data arrays
        self.x_data.append(msg.pose.pose.position.x)
        self.y_data.append(msg.pose.pose.position.y)
    
    def update_plot(self, frame):
        # Update the robot position plot
        self.ln.set_data(self.x_data, self.y_data)
        
        # If the goal is cancelled, remove the red marker
        if cancelled:
            self.goal_ln.set_data([], [])
        # If the goal exists, set the red marker
        elif goal is not None:
            self.goal_ln.set_data(goal.target_pose.pose.position.x, goal.target_pose.pose.position.y)
        else:
            self.goal_ln.set_data([], [])
        return self.ln, self.goal_ln

For visualizing the number of reached and cancelled goals, we can create another animation class:

In [ ]:
class GoalsVisualizer:
    def __init__(self):
        # Initialize the figure and axis
        self.fig, self.ax = plt.subplots()
        self.ax.set_title('Goals reached/cancelled')
        
        # Set up the initial values for the two bars
        self.reached = 0
        self.cancelled = 0
        
        # Set up the horizontal grid
        self.ax.grid(axis='y', color='grey', linestyle='-', alpha=0.5)
        
        # Set up the x-axis tick labels
        self.labels = ('Reached', 'Cancelled')
        self.x_pos = np.arange(len(self.labels))
        self.ax.set_xticks(self.x_pos)
        self.ax.set_xticklabels(self.labels)
        
        # Set up the y-axis range and ticks
        self.ax.set_ylim([0, 10])
        self.ax.set_yticks(np.arange(0, 11, 1))
        
        # Set up the bar chart
        self.bar_colors = ['green', 'red']
        self.bar_plot = self.ax.bar(self.x_pos,
                                    [self.reached, self.cancelled],
                                    align='center',
                                    color=self.bar_colors,
                                    width=0.2)
        
    def goals_callback(self, msg):
        # Get the number of reached and cancelled goals
        response = call_service()
        self.reached = response.reached
        self.cancelled = response.cancelled
        
    def update_plot(self, frame):
        # Update the values of the two bars
        self.green_val = np.random.randint(0, 100)
        self.red_val = np.random.randint(0, 100)
        
        # Update the heights of the bars
        for i, bar in enumerate(self.bar_plot):
            if i == 0:
                bar.set_height(self.reached)
            else:
                bar.set_height(self.cancelled)
                
            bar.set_color(self.bar_colors[i])
        
        return self.bar_plot

Now we need to initialize the node, subscriber, publisher and client:

In [ ]:
# Initialize the node
rospy.init_node('action_client_jupy')
# Publisher for the custom message
pub = rospy.Publisher("/pos_vel", Pos_vel, queue_size=1)
# Create the action client
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
# Wait for the server to be started
client.wait_for_server()
# Set the cancelled flag to false
cancelled = False
# Set the goal to None
goal = None

## Program execution
Run the following cell to be able to set the **desired position**

In [ ]:
display(widgets.HBox([x_widg, y_widg]))
display(widgets.HBox([send_button, cancel_button]))

Run the following cell to print the **current position** of the robot

In [ ]:
last_odom = 0
jr.subscribe('/odom', Odometry, clbk_odom)

Run the following cell to print the distance from the **closest obstacle**

In [ ]:
last_laser = 0
jr.subscribe('/scan', LaserScan, clbk_laser)

Run the following cell to plot the number of **cancelled goals** and the number of **reached goals**

In [ ]:
# Create the visualizer object
g_vis = GoalsVisualizer()
# Subscriber for the goals visualizer
sub_result = rospy.Subscriber('/reaching_goal/result', assignment_2_2022.msg.PlanningActionResult, g_vis.goals_callback)

g_ani = FuncAnimation(g_vis.fig, g_vis.update_plot, interval=1000)
plt.show(block=True)

Run the following cell to plot the **robot position** and the **goal position**

In [ ]:
# Create the visualizer object
pos_vis = PositionVisualizer()
# Subscriber for the odom topic
sub = rospy.Subscriber('/odom', Odometry, pos_vis.odom_callback)

pos_ani = FuncAnimation(pos_vis.fig, pos_vis.update_plot, init_func=pos_vis.plot_init)
plt.show(block=True)